# Tarea 7

## Instrucciones
Descarga los siguientes datos de Yelp en JSON. Estos son los metadatos del dataset.

Utiliza el contenedor de PySpark para responder las siguientes preguntas.

Nota: Invierte tiempo en esta tarea porque te va a servir para el proyecto parcial de la materia, tanto para ir practicando la notración de Pyspark como para manejar los datos.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yelp-dataset/Dataset_User_Agreement.pdf
/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json


## Instalar pyspark

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 5.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=4ac93648e763081699609f9e414f2e93f65c3c39523dd58f8d616d3e456d0a48
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# Importar Bibliotecas

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import time

Iniciar Session (Código de Enrique en Slack)

In [4]:
spark = SparkSession.builder \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/15 15:17:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Preguntas

### Toma el archivo review.json JSON y cuantífica cuánto pesa el archivo en disco.

#### **Pesa 5.34 GB**

Carga el JSON en Spark y cuantífica cuánto pesa el DataFramen en memoria RAM.

In [5]:
yelp_review = spark.read.json(path='/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json')

In [7]:
yelp = yelp_review.cache().select(yelp_review.columns)
size_in_bytes = yelp._jdf.queryExecution().optimizedPlan().stats().sizeInBytes() 
size_in_bytes

24/04/15 15:18:12 WARN CacheManager: Asked to cache already cached data.


5341868833

**5.34 GB de RAM**

### Guarda el DataFrame como parquet en disco y muestra cuánto pesa el archivo. Cómo se compara con el JSON crudo.

In [17]:
yelp_review.write.parquet(path='/kaggle/temp/',
                       mode='append',
                       compression='none')

**El parque guardado pesa 4.3 GB, un gigabyte menos que el json**

### Utiliza el DataFrame, optimiza el tipo de dato que hay en cada columna (i.e. Int32, Int64, Float32, Float64, String, Categorical) y guarda el nuevo DataFrame como parquet. Cuántifica cuánto pesa el DataFrame en memoria RAM y cuánto pesa en disco. Cómo se compara con el parquet crudo.

In [18]:
yelp_review.show()
yelp_review.printSchema()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

Estos son los cambios que se harían:
* business_id -> se queda igual
* cool -> boolean
* date -> timestamp
* funny -> integer
* review_id -> se queda igual
* stars -> integer
* text -> se queda igual
* useful -> integer
* user_id -> se queda igual

In [8]:
yelp_review = yelp_review.withColumn("cool", yelp_review["cool"].cast("boolean"))
yelp_review = yelp_review.withColumn("date", yelp_review["date"].cast("timestamp"))
yelp_review = yelp_review.withColumn("funny", yelp_review["funny"].cast("integer"))
yelp_review = yelp_review.withColumn("stars", yelp_review["stars"].cast("integer"))
yelp_review = yelp_review.withColumn("useful", yelp_review["useful"].cast("integer"))
yelp_review.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: boolean (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- funny: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: integer (nullable = true)
 |-- user_id: string (nullable = true)



In [9]:
yelp = yelp_review.cache().select(yelp_review.columns)
size_in_bytes = yelp._jdf.queryExecution().optimizedPlan().stats().sizeInBytes() 
size_in_bytes

4159026448

**4.16 GB en RAM**

### Utiliza el DataFrame optimizado, guarda en parquet una nueva versión del DataFrame y particionalo por fecha (date).

#### Nota: Para particionar por fecha hazlo por año y mes. Para hacer esto necesitas extraer el año y mes como columnas.

In [22]:
## Forma alternativa vista en:
## https://medium.com/@uzzaman.ahmed/pyspark-date-time-functions-a-comprehensive-guide-b250e92df264
# from pyspark.sql.functions import *
# yelp_review.select("business_id",
#                    "cool",
#                    year("date"), 
#                    month("date"),
#                    "funny",
#                    "review_id",
#                    "stars",
#                    "text",
#                    "useful",
#                    "user_id").show()

Particionando por fecha: año y mes

In [23]:
yelp_review_fecha = (
    yelp_review
    .withColumn("year", f.year(f.col("date")))
    .withColumn("month", f.month(f.col("date")))
    .select("business_id","cool", "year", "month", "funny", "review_id", "stars", "text", "useful", "user_id")
)

In [24]:
yelp_review_fecha.show()

+--------------------+-----+----+-----+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id| cool|year|month|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+-----+----+-----+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|false|2018|    7|    0|KU_O5udG6zpxOg-Vc...|    3|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...| true|2012|    1|    0|BiTunyQ73aT9WBnpR...|    5|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|false|2014|    2|    0|saUsX_uimxRlCVr67...|    3|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...| true|2015|    1|    0|AqPFMleE6RsU23_au...|    5|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...| true|2017|    1|    0|Sx8TMOWLNuJBWer-0...|    4|Cute interior and...|     1|bcjbaE6dDog4jkNY9...|
|04UD14gamNjLY0IDY...| t

In [25]:
(yelp_review_fecha
    .write.parquet(
    "fecha",
    mode="overwrite",
    partitionBy=["year","month"])
)

**Este parquet pesa 2.9 GB**

### Nota: Para particionar por ~ciudad (city)~ estado (state) será necesario que hagas un join con la tabla business.json.

In [26]:
yelp_business = spark.read.json(path='/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json') \
                      .selectExpr("business_id", "state")
yelp_business.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- state: string (nullable = true)



Haciendo el join

In [27]:
# Perform the join operation
joined_df = yelp_review.join(yelp_business, on="business_id")
joined_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: boolean (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- funny: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: integer (nullable = true)
 |-- user_id: string (nullable = true)
 |-- state: string (nullable = true)



### Otra versión por ~ciudad~ estado. 

In [28]:
(joined_df
    .write.parquet(
    "estado",
    mode="overwrite",
    partitionBy=["state"])
)

**El parquet pesa 2.8 GB**

### Otra por ~ciudad~ estado y fecha.

In [29]:
yelp_review_cyf = (
    joined_df
    .withColumn("year", f.year(f.col("date")))
    .withColumn("month", f.month(f.col("date")))
    .select("business_id","cool", "year", "month", "funny", "review_id", "stars", "text", "useful", "user_id", "state")
)

yelp_review_cyf.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: boolean (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- funny: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: integer (nullable = true)
 |-- user_id: string (nullable = true)
 |-- state: string (nullable = true)



In [30]:
(yelp_review_cyf
    .write.parquet(
    "cyf",
    mode="overwrite",
    partitionBy=["year","month","state"])
)

**El parquet pesa 4 GB**

## Ejecuta un query utilizando sobre la tabla filtrando por una de las ciudades y un años en particular. Registra el tiempo de ejecución. Aplica el query sobre

### la versión en JSON

In [31]:
reviews_raw = spark.read.json(path='/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json')

In [32]:
from pyspark.sql.types import * 
spark.udf.register('udf_month', lambda x: int(x[5:7]), IntegerType())

<function __main__.<lambda>(x)>

In [33]:
# Create temporary table for SQL querying
reviews_raw.createOrReplaceTempView("reviews")

# Reduce DataFrame to include only relevant variables
reviews = spark.sql('SELECT review_id, stars, text, user_id, business_id, useful FROM reviews')

# Update temporary table for SQL querying
reviews.createOrReplaceTempView("reviews")

reviews.show(5) # Look at first 5 entries

+--------------------+-----+--------------------+--------------------+--------------------+------+
|           review_id|stars|                text|             user_id|         business_id|useful|
+--------------------+-----+--------------------+--------------------+--------------------+------+
|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|mh_-eMZ6K5RLWhZyI...|XQfwVwDr-v0ZS3_Cb...|     0|
|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|OyoGAe7OKpv6SyGZT...|7ATYjTIgM3jUlt4UM...|     1|
|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|8g_iMtfSiwikVnbP2...|YjUWPpI6HXG530lwP...|     0|
|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|_7bHUi9Uuf5__HHc_...|kxX2SOes4o-D3ZQBk...|     1|
|Sx8TMOWLNuJBWer-0...|  4.0|Cute interior and...|bcjbaE6dDog4jkNY9...|e4Vwtrqf-wpJfwesg...|     1|
+--------------------+-----+--------------------+--------------------+--------------------+------+
only showing top 5 rows



In [34]:
start_time = time.time()

# Number of reviews (count of rows)
spark.sql('SELECT COUNT(*) AS review_count FROM reviews').show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------+
|review_count|
+------------+
|     6990280|
+------------+

--- 14.888036489486694 seconds ---


### la versión particionada por fecha y luego

In [35]:
# Create temporary table for SQL querying
yelp_review_fecha.createOrReplaceTempView("reviews")

# Reduce DataFrame to include only relevant variables
reviews = spark.sql('SELECT review_id, stars, text, user_id, business_id, useful FROM reviews')

# Update temporary table for SQL querying
reviews.createOrReplaceTempView("reviews")

reviews.show(5) # Look at first 5 entries

+--------------------+-----+--------------------+--------------------+--------------------+------+
|           review_id|stars|                text|             user_id|         business_id|useful|
+--------------------+-----+--------------------+--------------------+--------------------+------+
|KU_O5udG6zpxOg-Vc...|    3|If you decide to ...|mh_-eMZ6K5RLWhZyI...|XQfwVwDr-v0ZS3_Cb...|     0|
|BiTunyQ73aT9WBnpR...|    5|I've taken a lot ...|OyoGAe7OKpv6SyGZT...|7ATYjTIgM3jUlt4UM...|     1|
|saUsX_uimxRlCVr67...|    3|Family diner. Had...|8g_iMtfSiwikVnbP2...|YjUWPpI6HXG530lwP...|     0|
|AqPFMleE6RsU23_au...|    5|Wow!  Yummy, diff...|_7bHUi9Uuf5__HHc_...|kxX2SOes4o-D3ZQBk...|     1|
|Sx8TMOWLNuJBWer-0...|    4|Cute interior and...|bcjbaE6dDog4jkNY9...|e4Vwtrqf-wpJfwesg...|     1|
+--------------------+-----+--------------------+--------------------+--------------------+------+
only showing top 5 rows



In [36]:
start_time = time.time()

# Number of reviews (count of rows)
spark.sql('SELECT COUNT(*) AS review_count FROM reviews').show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------+
|review_count|
+------------+
|     6990280|
+------------+

--- 15.47508430480957 seconds ---


### sobre la versión particionada por fecha y lugar

In [37]:
# Create temporary table for SQL querying
yelp_review_cyf.createOrReplaceTempView("reviews")

# Reduce DataFrame to include only relevant variables
reviews = spark.sql('SELECT review_id, stars, text, user_id, business_id, useful FROM reviews')

# Update temporary table for SQL querying
reviews.createOrReplaceTempView("reviews")

reviews.show(5) # Look at first 5 entries

+--------------------+-----+--------------------+--------------------+--------------------+------+
|           review_id|stars|                text|             user_id|         business_id|useful|
+--------------------+-----+--------------------+--------------------+--------------------+------+
|KU_O5udG6zpxOg-Vc...|    3|If you decide to ...|mh_-eMZ6K5RLWhZyI...|XQfwVwDr-v0ZS3_Cb...|     0|
|BiTunyQ73aT9WBnpR...|    5|I've taken a lot ...|OyoGAe7OKpv6SyGZT...|7ATYjTIgM3jUlt4UM...|     1|
|saUsX_uimxRlCVr67...|    3|Family diner. Had...|8g_iMtfSiwikVnbP2...|YjUWPpI6HXG530lwP...|     0|
|AqPFMleE6RsU23_au...|    5|Wow!  Yummy, diff...|_7bHUi9Uuf5__HHc_...|kxX2SOes4o-D3ZQBk...|     1|
|Sx8TMOWLNuJBWer-0...|    4|Cute interior and...|bcjbaE6dDog4jkNY9...|e4Vwtrqf-wpJfwesg...|     1|
+--------------------+-----+--------------------+--------------------+--------------------+------+
only showing top 5 rows



In [38]:
start_time = time.time()

# Number of reviews (count of rows)
spark.sql('SELECT COUNT(*) AS review_count FROM reviews').show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------+
|review_count|
+------------+
|     6990280|
+------------+

--- 19.610512733459473 seconds ---


### De todos los queries que ejecutaste cuáles fueron más rápidos.

* version JSON: 
    14.888036489486694 seconds
* version particionada por fecha:
    15.47508430480957 seconds
* version particionada por fecha y lugar:
    19.610512733459473 seconds

**Los más rápidos fueron**

In [ ]:
# import shutil

# # Specify the path to the output directory
# output_directory = "/kaggle/working/"

# # Delete the entire output directory and its contents
# shutil.rmtree(output_directory)

# # Confirm deletion
# print("Output directory deleted successfully!")

# Bibliografia

* https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset/data
* https://taupe-sprinkles-b926d5.netlify.app/10-spark
* Demo 09 ("Quickstart: DataFrame" y "Optimizar el almacenamiento de datos de gran escala")
* https://www.kaggle.com/code/kamaljp/intro-setup-pyspark-yt-tutorial
* https://github.com/mdilmanian/PySpark-Yelp/blob/master/project.ipynb
